In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import scanpy as sc
import scipy as sp

import sys
import os
sys.path.append(os.path.abspath("./utility_functions"))

import rz_functions as rz
import rz_utility_spring as srz

from time import time

/home/ng136/miniconda3/envs/py36/lib/python3.6/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


python version: 3.6.10


In [2]:
project_dir = 'spring/tox_marie/'
plot_name = 'all_cells_200406_04h07'

In [3]:
# get cell filter:
cell_ix = np.loadtxt(project_dir+plot_name+'/cell_filter.txt',dtype=int)

# gene list
gene_list = np.loadtxt(project_dir+'/genes.txt',dtype=str)

True

In [7]:
adata = 'backups/tox_marie_all_cells_unnormalized_pca_umap_leiden_35358x22631_backup_200412_21h30.h5ad'

In [8]:
Eraw = adata.X

In [9]:
Eraw.shape

(35358, 22631)

# Load reference expression profiles in an AnnData object compatible with scanpy¶

In [10]:
prepath = 'Zilionis_Immunity_2019/'

In [11]:
# load count data
rzdata = sc.read_mtx(prepath+'GSE127465_mouse_counts_normalized_15939x28205.mtx.gz')

# load cell annotations
rzdata.obs = pd.read_csv(prepath+'GSE127465_mouse_cell_metadata_15939x12.tsv.gz',
            sep='\t',comment='#')

# load gene names
rzdata.var_names = np.loadtxt(prepath+'GSE127465_gene_names_mouse_28205.tsv.gz',dtype=str)

rzdata.shape

(15939, 28205)

In [12]:
#normalize immunity paper data to 10,000 counts per cell
sc.pp.normalize_per_cell(rzdata,counts_per_cell_after=1e4)


Transforming to str index.


In [13]:
cat0 = rz.centroids('Minor subset', rzdata).T

In [14]:
cat0.head()

,Mac1,DC1,DC3,Mono3,Mac2,N4,T2,Mono1,T3,T1,...,Mac3,pDC,NK cells,MonoDC,B cells,Mono2,N2,Mac4,Basophils,N3
0610007P14Rik,0.528133,0.260421,0.469878,0.187770,0.469109,0.190145,0.402685,0.264594,1.063999,0.474985,...,0.157427,1.194086,0.468520,0.475552,0.368709,0.374221,0.083965,0.785277,0.0,0.042854
0610009B22Rik,0.167399,0.544554,0.033153,0.263399,0.104697,0.154510,0.032572,0.099438,0.000000,0.030961,...,0.000000,0.274819,0.141834,0.382270,0.071297,0.138071,0.033403,0.155395,0.0,0.046007
0610009L18Rik,0.000000,0.000000,0.020329,0.011041,0.000000,0.030162,0.118733,0.007362,0.000000,0.026514,...,0.030877,0.075785,0.017998,0.000000,0.013612,0.010631,0.000000,0.000000,0.0,0.000000
0610009O20Rik,0.020461,0.027731,0.015273,0.065693,0.003815,0.029087,0.101459,0.049431,0.073788,0.038778,...,0.000000,0.150212,0.054022,0.000000,0.062836,0.000000,0.000000,0.059979,0.0,0.103472
0610010F05Rik,0.122463,0.015066,0.272357,0.025895,0.066593,0.058477,0.109913,0.078456,0.244052,0.045135,...,0.000000,0.058566,0.111800,0.078463,0.038325,0.084380,0.064667,0.116328,0.0,0.000000


In [15]:
pseudo = 0.1

cat = cat0 + pseudo

# Find common genes between the two datasets.
 Filtering on variable genes is also a possibility but I start by simply using all genes

In [16]:
Eraw = adata.X
print(type(Eraw))

gene_list = adata.var_names

<class 'scipy.sparse.csr.csr_matrix'>


In [17]:
# common genes
gmask = np.in1d(gene_list, cat.index)

# genes detected in the current dataset:
m2 = np.array(Eraw.sum(axis=0))[0]>0

# combine masks
gmask = gmask&m2


common_genes = gene_list[gmask]
print(len(gene_list),len(cat.index),len(common_genes))

22631 28205 15366


In [18]:
print(Eraw.shape)

(35358, 22631)


In [19]:
start = time()
bays = []
i = 0
step=5000
comment = 'tox_marie_all_cells'
for j in range(step,Eraw.shape[0]+step,step):
    
    # Eraw - sparse cells x gene matrix
    j = min(j,Eraw.shape[0])
    tmp_dense = pd.DataFrame(Eraw.T[gmask][:,i:j].todense())
    tmp_dense.index = np.array(gene_list)[gmask]
    
    bay = rz.bayesian_classifier(tmp_dense,cat.loc[common_genes])
    bays.append(bay)
    i0 = i
    i = j
    
    print('%.2f min.'%((time()-start)/60.))
    print('cells from %d to %d done'%(i0,j))

# conenate
bay = pd.concat(bays,axis=1)

# reset index
bay.columns = np.arange(bay.shape[1])

fname = 'backups/loglikelihoods_bay_classif_%s_%s'%(comment,rz.now())
print(fname)
rz.save_df(bay,fname)

1.57 min.
cells from 0 to 5000 done
3.14 min.
cells from 5000 to 10000 done
4.66 min.
cells from 10000 to 15000 done
6.20 min.
cells from 15000 to 20000 done
7.73 min.
cells from 20000 to 25000 done
9.29 min.
cells from 25000 to 30000 done
10.81 min.
cells from 30000 to 35000 done
10.92 min.
cells from 35000 to 35358 done
backups/loglikelihoods_bay_classif_tox_marie_all_cells_200408_00h46


# Add results to SPRING plot (to do)

In [20]:
# get profile with max likelihood
mostlikely = bay.idxmax().values
print(len(mostlikely))
print(len(cell_ix))

# use cell_ix to only select cell in the desired spring plot:
mostlikely = list(mostlikely[cell_ix])
mostlikely[:10]

35358
35358


['Mac1', 'N1', 'NK cells', 'Mono2', 'T1', 'DC3', 'N2', 'T2', 'N2', 'Mac1']

In [21]:
bay.head()

,0,1,2,3,4,5,6,7,8,9,...,35348,35349,35350,35351,35352,35353,35354,35355,35356,35357
Mac1,-1625.227902,-3757.041910,-2488.629885,-1773.142636,-2130.073564,-4817.528489,-814.193230,-2167.657116,-1430.989031,-1868.892945,...,-6140.197042,-4784.231735,-2186.351560,-1595.204412,-8069.469067,-949.878185,-3591.251020,-2504.314787,-4754.326110,-3506.113237
DC1,-1645.671513,-3962.437781,-2479.818125,-1763.858226,-2074.437328,-4738.976058,-865.885944,-2155.920275,-1509.025566,-1887.504846,...,-6126.002926,-4712.543523,-2171.966556,-1563.363786,-7946.930918,-940.568923,-3529.483387,-2460.491064,-4723.873380,-3642.771109
DC3,-1655.931918,-3918.906346,-2482.642368,-1781.096411,-2096.205709,-4436.082208,-858.909114,-2152.940119,-1491.280640,-1889.003279,...,-5747.414860,-4361.241097,-2103.602341,-1491.370985,-8071.792746,-897.652042,-3298.959111,-2334.382484,-4414.932476,-3661.789910
Mono3,-1638.799124,-3439.758871,-2509.675886,-1737.780949,-2169.695484,-4874.502357,-768.024848,-2164.290416,-1350.976760,-1881.067453,...,-6179.331332,-4863.620844,-2202.800982,-1614.986951,-8199.106613,-968.965623,-3693.223742,-2545.327729,-4818.562359,-3555.493445
Mac2,-1650.093455,-3898.382490,-2549.622862,-1818.787785,-2190.797110,-4998.181521,-839.774536,-2223.958431,-1467.777593,-1897.131875,...,-6369.762501,-4978.422503,-2258.133702,-1651.851662,-8236.201415,-984.533170,-3747.722232,-2586.727847,-4913.379565,-3587.726096
